In [ ]:
!pip install transformers --quiet
!pip install git+https://github.com/abheesht17/keras-nlp.git@gpt2-presets tensorflow tensorflow-text --upgrade --quiet

     |████████████████████████████████| 5.5 MB 5.3 MB/s 
     |████████████████████████████████| 7.6 MB 46.3 MB/s 
     |████████████████████████████████| 163 kB 69.9 MB/s 
     |████████████████████████████████| 578.0 MB 12 kB/s 
     |████████████████████████████████| 5.9 MB 45.7 MB/s 
     |████████████████████████████████| 1.7 MB 65.1 MB/s 
     |████████████████████████████████| 5.9 MB 40.0 MB/s 
     |████████████████████████████████| 438 kB 46.0 MB/s 


In [ ]:
import json
import os

import keras_nlp
import numpy as np

import tensorflow as tf
from tensorflow import keras

import torch

from transformers import AutoTokenizer, AutoModel

tf.__version__

'2.10.0'

In [ ]:
# Variables.

# Should be one of 124M, 355M, 774M, 1558M.
NUM_PARAMS = "774M"

In [ ]:
# Let's get the official OpenAI GPT-2 ckpt.

!wget https://raw.githubusercontent.com/openai/gpt-2/master/download_model.py
!python3 download_model.py $NUM_PARAMS

--2022-11-13 09:28:27--  https://raw.githubusercontent.com/openai/gpt-2/master/download_model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1075 (1.0K) [text/plain]
Saving to: ‘download_model.py’

download_model.py   100%[===================>]   1.05K  --.-KB/s    in 0s      

2022-11-13 09:28:27 (46.5 MB/s) - ‘download_model.py’ saved [1075/1075]

Fetching checkpoint: 1.00kit [00:00, 836kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 2.44Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 811kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 3.10Git [01:46, 29.0Mit/s]                                 
Fetching

In [ ]:
# GPT-2 paths.
extract_dir = f"/content/models/{NUM_PARAMS}"
merges_path = os.path.join(extract_dir, "vocab.bpe")
vocab_path = os.path.join(extract_dir, "encoder.json")
checkpoint_path = os.path.join(extract_dir, "model.ckpt")
config_path = os.path.join(extract_dir, "hparams.json")

In [ ]:
# n_vocab, n_ctx, n_embd, n_head, n_layer
with open(config_path, "r") as f:
    cfg = json.load(f)

cfg

{'n_vocab': 50257, 'n_ctx': 1024, 'n_embd': 1280, 'n_head': 20, 'n_layer': 36}

In [ ]:
vars = tf.train.list_variables(checkpoint_path)
weights = {}
for name, shape in vars:
    print(name, shape)
    weight = tf.train.load_variable(checkpoint_path, name)
    weights[name] = weight

model/h0/attn/c_attn/b [3840]
model/h0/attn/c_attn/w [1, 1280, 3840]
model/h0/attn/c_proj/b [1280]
model/h0/attn/c_proj/w [1, 1280, 1280]
model/h0/ln_1/b [1280]
model/h0/ln_1/g [1280]
model/h0/ln_2/b [1280]
model/h0/ln_2/g [1280]
model/h0/mlp/c_fc/b [5120]
model/h0/mlp/c_fc/w [1, 1280, 5120]
model/h0/mlp/c_proj/b [1280]
model/h0/mlp/c_proj/w [1, 5120, 1280]
model/h1/attn/c_attn/b [3840]
model/h1/attn/c_attn/w [1, 1280, 3840]
model/h1/attn/c_proj/b [1280]
model/h1/attn/c_proj/w [1, 1280, 1280]
model/h1/ln_1/b [1280]
model/h1/ln_1/g [1280]
model/h1/ln_2/b [1280]
model/h1/ln_2/g [1280]
model/h1/mlp/c_fc/b [5120]
model/h1/mlp/c_fc/w [1, 1280, 5120]
model/h1/mlp/c_proj/b [1280]
model/h1/mlp/c_proj/w [1, 5120, 1280]
model/h10/attn/c_attn/b [3840]
model/h10/attn/c_attn/w [1, 1280, 3840]
model/h10/attn/c_proj/b [1280]
model/h10/attn/c_proj/w [1, 1280, 1280]
model/h10/ln_1/b [1280]
model/h10/ln_1/g [1280]
model/h10/ln_2/b [1280]
model/h10/ln_2/g [1280]
model/h10/mlp/c_fc/b [5120]
model/h10/mlp/

In [ ]:
model = keras_nlp.models.GPT2.from_preset(
    "gpt2_large",
    load_weights=False,
)

In [ ]:
model.summary()

Model: "backbone"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 token_ids (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 token_embedding (Embedding)    (None, None, 1280)   64328960    ['token_ids[0][0]']              
                                                                                                  
 position_embedding (PositionEm  (None, None, 1280)  1310720     ['token_embedding[0][0]']        
 bedding)                                                                                         
                                                                                                  
 add (Add)                      (None, None, 1280)   0           ['token_embedding[0][0]', 

In [ ]:
model.get_layer("token_embedding").embeddings.assign(
    weights["model/wte"]
)
model.get_layer("position_embedding").position_embeddings.assign(
    weights["model/wpe"]
)

range_1 = (0, cfg["n_embd"])
range_2 = (cfg["n_embd"], 2 * cfg["n_embd"]) 
range_3 = (2 * cfg["n_embd"], 3 * cfg["n_embd"])


for i in range(model.num_layers):
    model.get_layer(f"transformer_layer_{i}")._self_attention_layer._query_dense.kernel.assign(
        weights[f"model/h{i}/attn/c_attn/w"][0, :, range_1[0]: range_1[1]].reshape((cfg["n_embd"], cfg["n_head"], -1))
    )
    model.get_layer(f"transformer_layer_{i}")._self_attention_layer._query_dense.bias.assign(
        weights[f"model/h{i}/attn/c_attn/b"][range_1[0]: range_1[1]].reshape((cfg["n_head"], -1))
    )
    model.get_layer(f"transformer_layer_{i}")._self_attention_layer._key_dense.kernel.assign(
        weights[f"model/h{i}/attn/c_attn/w"][0, :, range_2[0]: range_2[1]].reshape((cfg["n_embd"], cfg["n_head"], -1))
    )
    model.get_layer(f"transformer_layer_{i}")._self_attention_layer._key_dense.bias.assign(
        weights[f"model/h{i}/attn/c_attn/b"][range_2[0]: range_2[1]].reshape((cfg["n_head"], -1))
    )
    model.get_layer(f"transformer_layer_{i}")._self_attention_layer._value_dense.kernel.assign(
        weights[f"model/h{i}/attn/c_attn/w"][0, :, range_3[0]: range_3[1]].reshape((cfg["n_embd"], cfg["n_head"], -1))
    )
    model.get_layer(f"transformer_layer_{i}")._self_attention_layer._value_dense.bias.assign(
        weights[f"model/h{i}/attn/c_attn/b"][range_3[0]: range_3[1]].reshape((cfg["n_head"], -1))
    )


    model.get_layer(f"transformer_layer_{i}")._self_attention_layer._output_dense.kernel.assign(
        weights[f"model/h{i}/attn/c_proj/w"][0].reshape((cfg["n_head"], -1, cfg["n_embd"]))
    )
    model.get_layer(f"transformer_layer_{i}")._self_attention_layer._output_dense.bias.assign(
        weights[f"model/h{i}/attn/c_proj/b"]
    )


    model.get_layer(f"transformer_layer_{i}")._self_attention_layernorm.gamma.assign(
        weights[f"model/h{i}/ln_1/g"]
    )
    model.get_layer(f"transformer_layer_{i}")._self_attention_layernorm.beta.assign(
        weights[f"model/h{i}/ln_1/b"]
    )


    model.get_layer(f"transformer_layer_{i}")._feedforward_intermediate_dense.kernel.assign(
        weights[f"model/h{i}/mlp/c_fc/w"][0]
    )
    model.get_layer(f"transformer_layer_{i}")._feedforward_intermediate_dense.bias.assign(
        weights[f"model/h{i}/mlp/c_fc/b"]
    )


    model.get_layer(f"transformer_layer_{i}")._feedforward_output_dense.kernel.assign(
        weights[f"model/h{i}/mlp/c_proj/w"][0]
    )
    model.get_layer(f"transformer_layer_{i}")._feedforward_output_dense.bias.assign(
        weights[f"model/h{i}/mlp/c_proj/b"]
    )


    model.get_layer(f"transformer_layer_{i}")._feedforward_layernorm.gamma.assign(
        weights[f"model/h{i}/ln_2/g"]
    )
    model.get_layer(f"transformer_layer_{i}")._feedforward_layernorm.beta.assign(
        weights[f"model/h{i}/ln_2/b"]
    )


model.get_layer("layer_norm").gamma.assign(
    weights[f"model/ln_f/g"]
)

model.get_layer("layer_norm").beta.assign(
    weights[f"model/ln_f/b"]
)
pass

In [ ]:
# Define KerasNLP components.

bpe_tokenizer = keras_nlp.tokenizers.BytePairTokenizer(
    vocabulary=vocab_path,
    merges=merges_path,
    # sequence_length=cfg["n_ctx"]
)

# Define HF components.

hf_tokenizer = AutoTokenizer.from_pretrained("gpt2-large")
hf_model = AutoModel.from_pretrained("gpt2-large")
hf_model.eval()
pass

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

In [ ]:
# KerasNLP
token_ids = bpe_tokenizer(["the quick brown fox ran, galloped and jumped."])
padding_mask = token_ids != 0

inputs = {
    "token_ids": token_ids.to_tensor(),
    "padding_mask": padding_mask.to_tensor()
}

keras_nlp_output = model.predict(inputs)

# HF
hf_inputs = hf_tokenizer(["the quick brown fox ran, galloped and jumped."], return_tensors="pt")
hf_output = hf_model(**hf_inputs).last_hidden_state

1/1 [==============================] - 8s 8s/step


In [ ]:
keras_nlp_output[0, 0, :10], hf_output[0, 0, :10]

(array([-0.15592659, -0.15414183, -0.01744013, -0.23087016,  0.17047967,
        -0.25551534, -0.8162856 , -0.5639553 ,  2.109562  ,  0.24356467],
       dtype=float32),
 tensor([-0.1559, -0.1541, -0.0174, -0.2309,  0.1705, -0.2555, -0.8163, -0.5640,
          2.1096,  0.2436], grad_fn=<SliceBackward0>))

In [ ]:
np.mean(keras_nlp_output - hf_output.detach().numpy())

7.3238393e-09

In [ ]:
# Save GPT2 checkpoint.
model.save_weights("gpt2_large.h5")

In [ ]:
model2 = keras_nlp.models.GPT2.from_preset(
    "gpt2_large",
    load_weights=False,
)
model2.load_weights("gpt2_large.h5")

In [ ]:
token_ids = bpe_tokenizer(["the quick brown fox ran, galloped and jumped."])
padding_mask = token_ids != 0

inputs = {
    "token_ids": token_ids.to_tensor(),
    "padding_mask": padding_mask.to_tensor()
}

keras_nlp_output2 = model2.predict(inputs)

1/1 [==============================] - 8s 8s/step


In [ ]:
tf.reduce_mean(keras_nlp_output - keras_nlp_output2)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [ ]:
!md5sum gpt2_large.h5

67957cb3dfc9e965960dabe068811e1a  gpt2_large.h5


In [ ]:
# Check uploaded model once added to repo
model_cloud = keras_nlp.models.GPT2.from_preset(
    "gpt2_large",
    load_weights=True
)

3096768960/3096768960 [==============================] - 26s 0us/step


In [ ]:
# Same output from cloud model
keras_nlp_output_cloud = model_cloud.predict(inputs)
tf.reduce_mean(keras_nlp_output - keras_nlp_output_cloud)

1/1 [==============================] - 8s 8s/step


<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [ ]:
keras_nlp_output_cloud[0, 0, :10]

array([-0.15592659, -0.15414183, -0.01744013, -0.23087016,  0.17047967,
       -0.25551534, -0.8162856 , -0.5639553 ,  2.109562  ,  0.24356467],
      dtype=float32)